In [76]:
#import library
import numpy as np
import pandas as pd
import torch
from torch import nn
from sklearn.model_selection import train_test_split

In [77]:
#read excel file
df = pd.read_excel("heart.xltx")
df.head()

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0


In [78]:
df.shape

(1190, 12)

In [79]:
df.columns

Index(['age', 'sex', 'chest pain type', 'resting bp s', 'cholesterol',
       'fasting blood sugar', 'resting ecg', 'max heart rate',
       'exercise angina', 'oldpeak', 'ST slope', 'target'],
      dtype='object')

In [80]:
#search missing value
df.isnull().sum()

,0
age,0
sex,0
chest pain type,0
resting bp s,0
cholesterol,0
fasting blood sugar,0
resting ecg,0
max heart rate,0
exercise angina,0
oldpeak,0


In [81]:
#makign data and label
y1 = df["target"]
X1 = df.drop("target",axis=1)

In [82]:
#chaking data type
cols = X1.columns
for col in cols:
  col_type = X1[col].dtype
  print(f"column name: {col} | data type {col_type}")

column name: age | data type int64
column name: sex | data type int64
column name: chest pain type | data type int64
column name: resting bp s | data type int64
column name: cholesterol | data type int64
column name: fasting blood sugar | data type int64
column name: resting ecg | data type int64
column name: max heart rate | data type int64
column name: exercise angina | data type int64
column name: oldpeak | data type float64
column name: ST slope | data type int64


In [83]:
#converting to tensor
X = torch.tensor(X1.values).type(torch.float)
y = torch.tensor(y1.values).type(torch.float)

In [84]:
#making train and test data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)
X_test.shape

torch.Size([119, 11])

In [85]:
#makel deep learning model
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(11,16),
        nn.ReLU(),
        nn.Linear(16,32),
        nn.ReLU(),
        nn.Linear(32,16),
        nn.ReLU(),
        nn.Linear(16,1)
    )

  def forward(self,x):
    return self.model(x)

In [86]:
#making loss function and optimizer
model = Model()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)

In [87]:
#creating accuracy function
def accuracy(y1,y2):
  acc1 = torch.eq(y1,y1).sum().item()
  acc = acc1/len(y1)
  return acc*100

In [88]:
#training the model
torch.manual_seed(42)
epochs = 101
for epoch in range(epochs):
  model.train()
  y_logs = model(X_train).squeeze()
  y_pred = torch.round(torch.sigmoid(y_logs))
  loss = loss_fn(y_logs,y_train)
  acc = accuracy(y_pred,y_train)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if not epoch%10:
    print(f"epoch {epoch} | loss {loss} | accuracy {acc}%")

epoch 0 | loss 1.2579745054244995 | accuracy 100.0%
epoch 10 | loss 0.6483278274536133 | accuracy 100.0%
epoch 20 | loss 0.6475337147712708 | accuracy 100.0%
epoch 30 | loss 0.6470493078231812 | accuracy 100.0%
epoch 40 | loss 0.6468777656555176 | accuracy 100.0%
epoch 50 | loss 0.6466862559318542 | accuracy 100.0%
epoch 60 | loss 0.6465575098991394 | accuracy 100.0%
epoch 70 | loss 0.6464437246322632 | accuracy 100.0%
epoch 80 | loss 0.6463441252708435 | accuracy 100.0%
epoch 90 | loss 0.6461896896362305 | accuracy 100.0%
epoch 100 | loss 0.6461319327354431 | accuracy 100.0%


In [89]:
#testing the model
model.eval()
with torch.inference_mode():
  t_logs = model(X_test).squeeze()
  t_pred = torch.round(torch.sigmoid(t_logs))
  t_loss = loss_fn(t_logs,y_test)
  t_acc = accuracy(t_pred,y_test)
print(f"test loss {t_loss}")
print(f"test accuracy {t_acc}%")

test loss 0.6241510510444641
test accuracy 100.0%


In [90]:
#saving the model
from pathlib import Path
save_dir = Path("models")
save_dir.mkdir(parents=True,exist_ok=True)
path = save_dir/"model.pth"
torch.save(model.state_dict(),path)